# Creación de archivos .csv para poblar la base de datos

In [1]:
import pandas as pd
import numpy as np

import data_const as c
import utils as u
import relations as r


## Entidad Cliente

In [2]:
try:
    cliente = pd.read_csv(c.filepath_client)

except:
    print('Creando entidad Cliente')
    n_client = c.cardinality['client']
    
    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')

    names_list = u.get_names(data_names_masc, 
                             data_names_fem, 
                             data_surnames,
                            n_client,
                            1)
    # Names
    names = names_list[0]
    surnames_1 = names_list[1]
    surnames_2 = names_list[2]
    names_masc = names_list[3]
    names_fem = names_list[4]
    n_client, n_masc, n_fem = names_list[5]
    
    # Curps
    curp_masc = list(u.get_curps(names_masc,
                             surnames_1[0: n_masc],
                             surnames_2[0: n_masc],
                             sex='M'))
    curp_fem = list(u.get_curps(names_fem,
                            surnames_1[n_masc:],
                            surnames_2[n_masc:],
                            sex='F'))
    curps = curp_masc + curp_fem

    # Direcciones
    cdmx_data = pd.read_csv(c.filepath_raw + '/all_cdmx_data.csv')
    np.random.seed(1)
    to_select = np.random.choice(np.arange(0, cdmx_data.index[-1]), size=n_client, replace=True)
    cdmx_client = cdmx_data.iloc[to_select]
    cdmx_client.astype({'cp': 'float32'})
    
    unities = u.get_unity(curps)
    
    fac, ins, uni = u.get_fac_ins_uni(unities[0], unities[1], unities[2])
    
    # Email
    emails = u.get_mails(names,
                        surnames_1,
                        surnames_2,
                        curps,
                        fac,
                        ins,
                        uni)
    
    # Creamos y rellenamos
    cliente = pd.DataFrame(r.dict_to_df_cliente)

    cliente['curp'] = np.array(curps, dtype=str)
    cliente['nombre'] = np.array(names, dtype=str)
    cliente['a_paterno'] = np.array(surnames_1, dtype=str)
    cliente['a_materno'] = np.array(surnames_2, dtype=str)
    cliente['calle_num'] = np.array(cdmx_client['calle_num'], dtype=str)
    cliente['municipio'] = np.array(cdmx_client['alcaldia'], dtype=str)
    cliente['cp'] = np.array(cdmx_client['cp'], dtype=int)
    cliente['colonia'] = np.array(cdmx_client['colonia'], dtype=str)
    cliente['estado'] = str(c.address)
    cliente['tel_casa'] = np.array(u.get_tels(n_client), dtype=int)
    cliente['tel_celular'] = np.array(u.get_tels(n_client), dtype=int)
    cliente['fotografia'] = np.array(u.get_photos(curps), dtype=str)
    cliente['es_alumno'] = np.array(unities[0], dtype=bool)
    cliente['es_academico'] = np.array(unities[1], dtype=bool)
    cliente['es_trabajador'] = np.array(unities[2], dtype=bool)
    cliente['facultad'] = np.array(fac,dtype=str)
    cliente['instituto'] = np.array(ins, dtype=str)
    cliente['unidad'] = np.array(uni, dtype=str)
    cliente['email'] = np.array(emails, dtype=str)
    
    # Verificamos que todos los curps sean distintos
    assert len(set(list(cliente['curp']))) == n_client
    
    # Escribimos CSV
    
    cliente.to_csv(c.filepath_client, index=False)

Creando entidad Cliente


In [3]:
cliente

,curp,nombre,a_paterno,a_materno,calle_num,municipio,cp,colonia,estado,tel_casa,tel_celular,email,fotografia,facultad,instituto,unidad,es_alumno,es_academico,es_trabajador
0,HEAJ040810MVSHNX83,JOSE LUIS,HEREDIA,ABAD,Angel R Cavada Mz 57 L 42,Álvaro Obregón,15000,Jalalpa Grande,Cuidad de México,5523062474,5511826602,joseabad04@ciencias.unam.mx,proyecto/data/photos/photo_HEAJ040810MVSHNX83.jpg,"Facultad de Ciencias, CU",No aplica,No aplica,True,False,False
1,COMM020318MHDSFZ49,ALBERTO,CONDE,MONTELLANO,Pacifico 274 Estac 14,Coyoacán,43500,Rosedal,Cuidad de México,5632595970,5583367079,albertoconde02@economia.unam.mx,proyecto/data/photos/photo_COMM020318MHDSFZ49.jpg,"Facultad de Economía, CU",No aplica,No aplica,True,False,False
2,GOCF550318MQNFBX74,JOSE,GOMEZ,CANSECO,"Avd. PACIFICO, 350 Dpto. E-001",Coyoacán,43300,EL ROSEDAL,Cuidad de México,5585105481,5518761766,josegomez55@dgp.unam.mx,proyecto/data/photos/photo_GOCF550318MQNFBX74.jpg,No aplica,No aplica,"Rectoría, CU",False,False,True
3,GALJ040301MCOHIV38,JORGE,GARCIA,LA,Cartzo 119 Local a,Cuauhtémoc,64000,Sta Maria la Rivera,Cuidad de México,5515646782,5549011342,jorgela04@psicologia.unam.mx,proyecto/data/photos/photo_GALJ040301MCOHIV38.jpg,"Facultad de Psicología, CU","Instituto de Biolgía, CU",No aplica,True,True,False
4,ZUSJ630322MTGFPW95,MANUEL,ZUÑIGA,SANTOS,Camelia M-7 L-9,Álvaro Obregón,17400,San Clemente,Cuidad de México,5595988134,5574091918,manuelsantos63@deporte.unam.mx,proyecto/data/photos/photo_ZUSJ630322MTGFPW95.png,No aplica,No aplica,"Estadio Olímpico, CU",False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,GACG840506FQPXTY06,MARGARITA,GARCIA,CONTRERAS,Camino a Sta. Cruz Num. 53,Iztapalapa,97900,Lomas de San Lorenzo,Cuidad de México,5550573398,5650758691,margaritagarcia84@fisica.unam.mx,proyecto/data/photos/photo_GACG840506FQPXTY06.jpg,No aplica,"Instituto de Física, CU",No aplica,False,True,False
9996,AGDG020212FZBHXL62,GABRIELA,AGUILERA,DEOLARTE,"Prv. EMPERADOR, MZ-1 LT-38",Magdalena Contreras,10369,TIERRA UNIDA,Cuidad de México,5545203874,5617192061,gabrieladeolarte02@odontologia.unam.mx,proyecto/data/photos/photo_AGDG020212FZBHXL62.png,"Facultad de Odontología, CU",No aplica,No aplica,True,False,False
9997,GUCG040914FMDCKI42,MARIA DE LOUR,GUTIERREZ,CARDENAS,Darwin 48,Miguel Hidalgo,11590,Anzures,Cuidad de México,5681858093,5557246707,mariacardenas04@quimica.unam.mx,proyecto/data/photos/photo_GUCG040914FMDCKI42.jpg,"Facultad de Química, CU",No aplica,No aplica,True,False,False
9998,MOAG620529FTMOXC77,LAURA ESTELA,MOJICA,ALVAREZ,"2DO AND 2 DE ABRIL, MZ-5 LT-3-A",Magdalena Contreras,10900,PUEBLO SAN NICOLAS TOTOLAPAN,Cuidad de México,5507041290,5537059724,lauramojica62@biologicas.unam.mx,proyecto/data/photos/photo_MOAG620529FTMOXC77.jpg,No aplica,"Instituto de Biolgía, CU",No aplica,False,True,False


## Entidad Socio

In [4]:
try:
    socio = pd.read_csv(c.filepath_partner)

except:
    print('Creando entidad Socio...')
    
    n_socio = c.cardinality['partner']
    
    # nombres
    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')

    names_list = u.get_names(data_names_masc, 
                             data_names_fem, 
                             data_surnames, 
                             n_socio, 
                             2)
    
    # Names
    names = names_list[0]
    surnames_1 = names_list[1]
    surnames_2 = names_list[2]
    names_masc = names_list[3]
    names_fem = names_list[4]
    _, n_masc, n_fem = names_list[5]
    
    # Curps
    curp_masc = list(u.get_curps(names_masc,
                             surnames_1[0: n_masc],
                             surnames_2[0: n_masc],
                             sex='M', socio=True))
    curp_fem = list(u.get_curps(names_fem,
                            surnames_1[n_masc:],
                            surnames_2[n_masc:],
                            sex='F', socio=True))
    curps = curp_masc + curp_fem
    rfcs = np.array(u.get_rfc(curps), dtype=str)
    
    # Direcciones
    cdmx_data = pd.read_csv(c.filepath_raw + '/all_cdmx_data.csv')
    np.random.seed(2)
    to_select = np.random.choice(np.arange(0, cdmx_data.index[-1]), size=n_socio, replace=True)
    cdmx_socio = cdmx_data.iloc[to_select]
    cdmx_socio.astype({'cp': 'float32'})
    
    ingreso, egreso = u.get_in_out(n_socio)
    
    # Dueños y choferes
    choferes, duenios = u.get_due_cho(n_socio)
    
    # Creamos y rellenamos
    socio = pd.DataFrame(r.dict_to_df_socio)
    
    socio['id_socio'] = np.array(['s-'+f'{id}'.zfill(4) for id in range(1, n_socio+1)], dtype=str)
    socio['nombre'] = np.array(names, dtype=str)
    socio['a_paterno'] = np.array(surnames_1, dtype=str)
    socio['a_materno'] = np.array(surnames_2, dtype=str)
    socio['rfc'] = rfcs
    socio['num_licencia'] = np.array(u.get_lic(choferes), dtype=str)
    socio['calle_num'] = np.array(cdmx_socio['calle_num'], dtype=str)
    socio['municipio'] = np.array(cdmx_socio['alcaldia'], dtype=str)
    socio['cp'] = np.array(cdmx_socio['cp'], dtype=int)
    socio['colonia'] = np.array(cdmx_socio['colonia'], dtype=str)
    socio['estado'] = str(c.address)
    socio['tel_celular'] = np.array(u.get_tels(n_socio), dtype=int)
    socio['email'] = np.array(u.get_simple_mail(names), dtype=str)
    socio['fotografia'] = np.array(u.get_photos(rfcs), dtype=str)
    socio['ingreso'] = ingreso
    socio['egreso'] = egreso
    socio['es_chofer'] = np.array(choferes, dtype=bool)
    socio['es_duenio'] = np.array(duenios, dtype=bool)
    
    # Escribimos CSV
    socio.to_csv(c.filepath_partner, index=False)
    

Creando entidad Socio...


In [5]:
socio

,id_socio,nombre,a_paterno,a_materno,rfc,num_licencia,calle_num,colonia,cp,municipio,estado,tel_celular,email,fotografia,ingreso,egreso,es_chofer,es_duenio
0,s-0001,LUIS ENRIQUE,FLORES,SOSA,FLSJ800122MLR8,S53745449,Zapote Secc.2 Edif.centauri 139 202-a,Romulo Sanchez Mireles,14070,Tlalpan,Cuidad de México,5522353121,luis777@hotmail.com,proyecto/data/photos/photo_FLSJ800122MLR8.jpg,2021-10-15,NaN,True,True
1,s-0002,HUMBERTO,HERNANDEZ,CAMACHO,HECM830709MCY3,T05186522,Froa#o 34 B 202,Valle Gomez,15210,Venustiano Carranza,Cuidad de México,5565590412,humberto643@gmail.com,proyecto/data/photos/photo_HECM830709MCY3.jpg,2011-01-18,NaN,True,False
2,s-0003,RODOLFO,GUERRERO,GUTIERREZ,GUGF810720MRB7,No aplica,Rio Usumacinta 33 Mza. 10 Lt. 14,Ampl. Lomas de San Bernabe,10300,Magdalena Contreras,Cuidad de México,5625717050,rodolfo709@gmail.com,proyecto/data/photos/photo_GUGF810720MRB7.jpg,2015-05-19,NaN,False,True
3,s-0004,RUBEN,VILLANUEVA,MEZA,VIMJ810724MHV3,B28290500,Halacho Lt 13 a Mz 806,San Nicolas Totolapan,14100,Tlalpan,Cuidad de México,5520305069,ruben946@gmail.com,proyecto/data/photos/photo_VIMJ810724MHV3.jpg,2018-02-10,2018-07-30,True,False
4,s-0005,RAFAEL,SAN,GASPAR,SAGJ780326MUR1,H82224922,Edif 6 Ent D Depto 203,Unidad Lindavista Vallejo,77200,Gustavo A. Madero,Cuidad de México,5501206771,rafael610@hotmail.com,proyecto/data/photos/photo_SAGJ780326MUR1.jpg,2016-05-14,NaN,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,s-0196,LUZ DEL CARMEN,HERNANDEZ,DOMINGUEZ,HEDG830428FKS7,No aplica,"Eje 1 PONIENTE PROL GUERRERO, 340 Edif. ALLEND...",NONOALCO TLATELOLCO,69000,Cuauhtémoc,Cuidad de México,5577105502,luz615@gmail.com,proyecto/data/photos/photo_HEDG830428FKS7.png,2014-04-11,NaN,False,True
196,s-0197,TANIA KARINA,LOPEZ,ZAVALA,LOZD830713FNT0,Q04984366,Esperanza S/n,Santo Domingo,21400,Azcapotzalco,Cuidad de México,5618593478,tania377@hotmail.com,proyecto/data/photos/photo_LOZD830713FNT0.png,2010-05-19,NaN,True,False
197,s-0198,RICARDA,RIVERA,RAMIREZ,RIRI830122FKK9,N77666450,Nezahualcoyotl Z 5 Mz 132 Lt 26,Sta Ma Aztahuacan,95000,Iztapalapa,Cuidad de México,5661081736,ricarda436@yahoo.com,proyecto/data/photos/photo_RIRI830122FKK9.png,2019-01-12,NaN,True,False
198,s-0199,MARIA ALEJANDRA,FLORES,AGUILAR,FLAA870620FZT0,P54783204,Aguascalientes Z 1 Mz 69 Lt 4,Chalma de Guadalupe,72100,Gustavo A. Madero,Cuidad de México,5618138031,maria529@gmail.com,proyecto/data/photos/photo_FLAA870620FZT0.png,2015-02-28,NaN,True,False


## Entidad Vehículo

In [6]:
try:
    vehicle = pd.read_csv(c.filepath_vehicle)

except:
    print('Creando entidad Vehiculo...')
    
    n_vehicle = c.cardinality['vehicle']
    poss_duenios = list(socio['id_socio'][socio.index[socio['es_duenio'] == True].tolist()])
    info_vehicles = u.get_vehicle_info(n_vehicle)
    info_aseg = u.get_aseg_info(n_vehicle)
    activos, razones = u.get_raz(n_vehicle)
    
    # Creamos y rellenamos
    vehicle = pd.DataFrame(r.dict_to_df_vehiculo)
    
    vehicle['num_economico'] = np.array(['eco-'+f'{i}'.zfill(4) for i in range(1, n_vehicle + 1)], dtype=str)
    vehicle['id_socio'] = np.array(u.get_duenios(poss_duenios, n_vehicle), dtype=str)
    vehicle['marca'] = np.array(info_vehicles[0], dtype=str)
    vehicle['modelo'] = np.array(info_vehicles[1], dtype=str)
    vehicle['anio'] = np.array(info_vehicles[2], dtype=int)
    vehicle['tipo_conduccion'] = np.array(info_vehicles[3], dtype=str)
    vehicle['num_cilindros'] = np.array(info_vehicles[4], dtype=int)
    vehicle['pasajeros'] = np.array(info_vehicles[5], dtype=int)
    vehicle['puertas'] = np.array(info_vehicles[6], dtype=int)
    vehicle['combustible'] = np.array(info_vehicles[7], dtype=str)
    vehicle['refaccion'] = np.array(info_vehicles[8], dtype=str)
    vehicle['aseguradora'] = np.array(info_aseg[0], dtype=str)
    vehicle['tipo_seg'] = np.array(info_aseg[1], dtype=str)
    vehicle['vigencia_seg'] = info_aseg[2]
    vehicle['activo'] = np.array(activos, dtype=bool)
    vehicle['razon'] = np.array(razones, dtype=str)

    # Escribimos CSV
    vehicle.to_csv(c.filepath_vehicle, index=False)
    
    

Creando entidad Vehiculo...


In [7]:
vehicle

,num_economico,id_socio,marca,modelo,anio,tipo_conduccion,num_cilindros,pasajeros,puertas,combustible,refaccion,aseguradora,tipo_seg,vigencia_seg,activo,razon
0,eco-0001,s-0017,FORD,Fiesta,2017,Cambio automático,4,4,4,Motor diesel,True,BBVA,Básica,2022-06-13,True,No aplica
1,eco-0002,s-0160,FORD,Figo,2010,Cambio manual,6,3,4,Motor diesel,True,BBVA,Básica,2022-10-27,True,No aplica
2,eco-0003,s-0121,CHEVROLET,Aveo,2013,Cambio automático,6,3,5,Motor diesel,False,BBVA,Básica,2022-12-15,True,No aplica
3,eco-0004,s-0099,NISSAN,Tsuru,2003,Cambio automático,4,4,4,Motor gasolina,True,BBVA,Básica,2022-06-24,True,No aplica
4,eco-0005,s-0116,NISSAN,Tsuru,2003,Cambio automático,4,4,4,Motor gasolina,True,BBVA,Básica,2021-02-15,True,No aplica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,eco-0496,s-0096,CHEVROLET,Aveo,2016,Cambio automático,4,3,5,Motor gasolina,True,BBVA,Limitada,2021-02-28,False,Accidente
496,eco-0497,s-0098,NISSAN,Versa,2008,Cambio automático,4,4,4,Motor gasolina,True,BBVA,Básica,2022-05-31,True,No aplica
497,eco-0498,s-0066,NISSAN,Tsuru,2001,Cambio automático,4,3,5,Motor gasolina,True,BBVA,Básica,2022-07-19,True,No aplica
498,eco-0499,s-0133,CHEVROLET,Onix,2019,Cambio automático,6,4,5,Motor gasolina,True,GNP,Plus,2021-01-18,True,No aplica


## Entidad Viaje

In [8]:
try:
    travel = pd.read_csv(c.filepath_travel)

except:
    print('Creando entidad Viaje...')
    
    n_travel = c.cardinality['travel']
    poss_ch = list(socio['id_socio'][socio.index[socio['es_chofer'] == True].tolist()])
    distances, times = u.get_dist_time(n_travel)
    
    # Creamos y rellenamos
    travel = pd.DataFrame(r.dict_to_df_viaje)
    
    travel['id_viaje'] = np.array(['v-' + f'{i}'.zfill(6) for i in range(1,n_travel+1)], dtype=str)
    travel['id_socio'] = np.random.choice(poss_ch, size=n_travel)
    travel['num_economico'] = np.random.choice(vehicle['num_economico'].tolist(), size=n_travel)
    travel['distancia'] = np.array(distances, dtype=float)
    travel['tiempo'] = np.array(times, dtype=int)
    travel['fecha'] = np.array([u.get_date(lower=2021)[0] for i in range(n_travel)], dtype=str)
    
    # Escribimos CSV
    travel.to_csv(c.filepath_travel, index=False)
    

Creando entidad Viaje...


In [9]:
travel

,id_viaje,id_socio,num_economico,distancia,tiempo,fecha
0,v-000001,s-0033,eco-0206,6.37,40,2021-05-31
1,v-000002,s-0120,eco-0307,1.56,10,2021-07-31
2,v-000003,s-0110,eco-0222,1.62,12,2021-11-18
3,v-000004,s-0062,eco-0292,0.69,10,2021-04-30
4,v-000005,s-0148,eco-0400,3.88,10,2021-07-25
...,...,...,...,...,...,...
24995,v-024996,s-0016,eco-0212,13.90,45,2021-06-18
24996,v-024997,s-0088,eco-0486,1.11,6,2021-02-20
24997,v-024998,s-0181,eco-0273,0.90,11,2021-01-18
24998,v-024999,s-0140,eco-0372,1.86,8,2021-06-28


## Entidad Infracción

In [10]:
try:
    infraction = pd.read_csv(c.filepath_infraction)

except:
    print('Creando entidad Infracción...')
    
    n_infraction = c.cardinality['infraction']
    infractores, nums, fechas = u.get_infrac(travel, n_infraction)
    
    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')
    oficiales_ = u.get_names(data_names_masc,
                           data_names_fem,
                           data_surnames,
                           20,
                           seed=123)
    ofi_names = oficiales_[0]
    ofi_sur1 = oficiales_[1]
    ofi_sur2 = oficiales_[2]
    oficiales = [f'{ofi_names[i]} {ofi_sur1[i]} {ofi_sur2[i]}' for i in range(20)]
    num, cp, calle = u.get_infrac_place(n_infraction)
    
    # Creamos y rellenamos
    infraction = pd.DataFrame(r.dict_to_df_infraccion)
    
    infraction['id_infraccion'] = np.array(['i-' + f'{i}'.zfill(4) for i in range(1, n_infraction+1)], dtype=str)
    infraction['id_socio'] = infractores
    infraction['num_economico'] = nums
    infraction['monto_pagar'] = [round(abs(1000.0 + np.random.normal(500, 100)), 2) for i in range(1, n_infraction+1)]
    infraction['agente'] = np.random.choice(oficiales, size=n_infraction)
    infraction['fecha'] = fechas
    infraction['hora'] = [u.get_time() for i in range(n_infraction)]
    infraction['municipio'] = num
    infraction['cp'] = cp
    infraction['calle'] = calle
    
    # Escribimos CSV
    infraction.to_csv(c.filepath_infraction, index=False)

Creando entidad Infracción...


/Users/ivanreyes/python_envs/py3.8_env/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [11]:
infraction

,id_infraccion,id_socio,num_economico,monto_pagar,agente,fecha,hora,municipio,cp,calle
0,i-0001,s-0101,eco-0012,1441.71,MARIA DEL PI LOPEZ LOZANO,2021-07-29,11:41:14,Coyoacán,66455,Circuito escolar
1,i-0002,s-0052,eco-0080,1629.94,CESAR TOVAR CASTAÑEDA,2021-06-22,08:08:43,Coyoacán,66455,Circuito escolar
2,i-0003,s-0194,eco-0327,1557.39,EDILBERTO VERGARA BARRADAS,2021-07-24,06:25:16,Coyoacán,66455,Circuito escolar
3,i-0004,s-0006,eco-0198,1422.70,ZENAIDA ESQUIVEL URIBE,2021-01-21,09:10:20,Coyoacán,66455,Circuito escolar
4,i-0005,s-0101,eco-0113,1547.53,CARLOS RODRIGO ALVAREZ NAVA,2021-10-14,19:22:26,Coyoacán,66455,Circuito escolar
...,...,...,...,...,...,...,...,...,...,...
95,i-0096,s-0053,eco-0378,1375.64,ROSAURA CHANONA REBOLLAR,2021-08-26,18:33:32,Coyoacán,66455,Circuito escolar
96,i-0097,s-0193,eco-0289,1466.74,EDILBERTO VERGARA BARRADAS,2021-11-03,21:42:19,Coyoacán,66455,Circuito escolar
97,i-0098,s-0049,eco-0220,1588.63,MARGARITA ACOSTA TAPIA,2021-09-11,08:57:45,Coyoacán,66455,Circuito escolar
98,i-0099,s-0175,eco-0290,1537.28,ISMAEL TORRES PACHECO,2021-02-20,06:38:53,Coyoacán,66455,Circuito escolar


## Entidad Abordar

In [12]:
try:
    board = pd.read_csv(c.filepath_board)

except:
    print('Creando entidad Abordar...')
    
    curps = cliente['curp']
    ids_viajes = travel['id_viaje']
    curps, viajes = u.get_abordaje(curps, ids_viajes)
    
    # Creamos y rellenamos
    board = pd.DataFrame(r.dict_to_df_abordar)
    
    board['curp'] = curps
    board['id_viaje'] = viajes
    
    board.to_csv(c.filepath_board, index=False)
    
    board = pd.read_csv(c.filepath_board)
    origenes, destinos = u.get_origen_destino(board, cliente)
    
    board['origen'] = origenes
    board['destino'] = destinos
    
    assert len(board['curp'].unique()) == c.cardinality['client']
    
    # Escribimos CSV
    board.to_csv(c.filepath_board, index=False)
    
    

Creando entidad Abordar...


In [13]:
board

,curp,id_viaje,origen,destino
0,MAIC021103MBDDJT60,v-000001,Norte 82 5905,"Facultad de Ciencias, CU"
1,TOSO730210MJPGVB31,v-000002,Aztecas 53 Acc a,"Tienda UNAM, CU"
2,COLA050409MCIZAI69,v-000002,Div. del Norte Num. 5234 Bloque F 203,"Facultad de Arquitectura, CU"
3,LOSM820913FCYYXE19,v-000003,Jose Vasconcelos 39 Edif.13-h.depto 001,"CEPE, CU"
4,RIHL570206MOFWPV80,v-000003,Av 477 26,"Archivo General, CU"
...,...,...,...,...
64928,ZACE010714MCUNBW42,v-024999,"Facultad de Derecho, CU",8a Cda Jesus Romero Flores 5
64929,NIHF070312MOTOMU79,v-024999,"Facultad de Ingeniería, CU",2 CJON NOGAL 25
64930,RABD020104FJJLZW89,v-024999,"Facultad de Contaduría y Administración, CU","Cl. LAGO ZURICH, 243 Edif. M-1 ALFA, Sec. SING..."
64931,QUSD050826FMKYZG75,v-025000,"Facultad de Psicología, CU","Cl. AVENIDA TECAMACHALCO, 49 Dpto. 201"


## Entidad Programar

In [14]:
try:
    schedule = pd.read_csv(c.filepath_schedule)

except:
    print('Creando entidad Programar...')
    programar_, viaje_, abordar_ = u.add_redondos(board, travel, cliente, socio, vehicle)
    
    # Creamos y rellenamos
    schedule = pd.DataFrame(r.dict_to_df_programar)
    
    # Creamos programar
    schedule['curp'] = programar_[0]
    schedule['id_viaje'] = programar_[1]
    schedule['h_entrada'] = programar_[2]
    schedule['h_salida'] = programar_[3]
    schedule['redondo'] = programar_[4]
    
    schedule.to_csv(c.filepath_schedule, index=False)
    
    # Actualizamos las demás tablas
    # Viaje
    viaje_app = pd.DataFrame({'id_viaje': viaje_[0],
                            'id_socio': viaje_[1],
                            'num_economico': viaje_[2],
                            'distancia': viaje_[3],
                            'tiempo': viaje_[4],
                            'fecha': viaje_[5]})
    
    #travel.append(viaje_app, ignore_index=True)

    pd.concat([travel,viaje_app]).to_csv(c.filepath_travel, index=False)
    
    # Abordar
    abordar_app = pd.DataFrame({'curp': abordar_[0],
                               'id_viaje': abordar_[1],
                               'origen': abordar_[2],
                               'destino': abordar_[3]})
    
    #board.append(abordar_app, ignore_index=True)
    
    pd.concat([board, abordar_app]).to_csv(c.filepath_board, index=False)


Creando entidad Programar...


In [15]:
travel

,id_viaje,id_socio,num_economico,distancia,tiempo,fecha
0,v-000001,s-0033,eco-0206,6.37,40,2021-05-31
1,v-000002,s-0120,eco-0307,1.56,10,2021-07-31
2,v-000003,s-0110,eco-0222,1.62,12,2021-11-18
3,v-000004,s-0062,eco-0292,0.69,10,2021-04-30
4,v-000005,s-0148,eco-0400,3.88,10,2021-07-25
...,...,...,...,...,...,...
24995,v-024996,s-0016,eco-0212,13.90,45,2021-06-18
24996,v-024997,s-0088,eco-0486,1.11,6,2021-02-20
24997,v-024998,s-0181,eco-0273,0.90,11,2021-01-18
24998,v-024999,s-0140,eco-0372,1.86,8,2021-06-28


In [16]:
viaje_app

,id_viaje,id_socio,num_economico,distancia,tiempo,fecha
0,v-025001,s-0022,eco-0266,2.25,6,2021-04-24
1,v-025002,s-0160,eco-0384,2.42,8,2021-09-26
2,v-025003,s-0005,eco-0381,2.42,8,2021-09-26
3,v-025004,s-0159,eco-0035,2.08,8,2021-03-31
4,v-025005,s-0131,eco-0450,2.07,8,2021-03-31
...,...,...,...,...,...,...
1972,v-026973,s-0065,eco-0360,0.29,10,2021-01-17
1973,v-026974,s-0170,eco-0476,0.29,10,2021-01-17
1974,v-026975,s-0193,eco-0294,0.84,10,2021-01-12
1975,v-026976,s-0162,eco-0102,0.84,10,2021-01-12


In [17]:
schedule

,curp,id_viaje,h_entrada,h_salida,redondo
0,HECA060227MYKFAQ22,v-025001,07:00:00,NaN,False
1,HECA060227MYKFAQ22,v-025002,07:00:00,NaN,True
2,HECA060227MYKFAQ22,v-025003,NaN,15:00:00,True
3,HECA060227MYKFAQ22,v-025004,07:00:00,NaN,True
4,HECA060227MYKFAQ22,v-025005,NaN,15:00:00,True
...,...,...,...,...,...
1972,UCAA041114MBENXG74,v-026973,08:00:00,NaN,True
1973,UCAA041114MBENXG74,v-026974,NaN,15:00:00,True
1974,UCAA041114MBENXG74,v-026975,08:00:00,NaN,True
1975,UCAA041114MBENXG74,v-026976,NaN,15:00:00,True


In [18]:
abordar_app

,curp,id_viaje,origen,destino
0,HECA060227MYKFAQ22,v-025001,Rublos 10,"Facultad de Química, CU"
1,HECA060227MYKFAQ22,v-025002,Rublos 10,"Facultad de Química, CU"
2,HECA060227MYKFAQ22,v-025003,"Facultad de Química, CU",Rublos 10
3,HECA060227MYKFAQ22,v-025004,Rublos 10,"Facultad de Química, CU"
4,HECA060227MYKFAQ22,v-025005,"Facultad de Química, CU",Rublos 10
...,...,...,...,...
1972,UCAA041114MBENXG74,v-026973,"Avd. AZTECAS, 215 Ent. C, Dpto. 416","Escuela Nacional de Trabajo Social, CU"
1973,UCAA041114MBENXG74,v-026974,"Escuela Nacional de Trabajo Social, CU","Avd. AZTECAS, 215 Ent. C, Dpto. 416"
1974,UCAA041114MBENXG74,v-026975,"Avd. AZTECAS, 215 Ent. C, Dpto. 416","Escuela Nacional de Trabajo Social, CU"
1975,UCAA041114MBENXG74,v-026976,"Escuela Nacional de Trabajo Social, CU","Avd. AZTECAS, 215 Ent. C, Dpto. 416"
